In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import torch
from sklearn.metrics import classification_report
import json
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("/data/npl/ICEK/VACNIC/src/data/assest/ViHateT5-base-HSD")
model = AutoModelForSeq2SeqLM.from_pretrained("/data/npl/ICEK/VACNIC/src/data/assest/ViHateT5-base-HSD")
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [2]:
import re
EXPLICIT_SLURS = {
    # ── Từ đơn ──
    "đm", "dm", "đmm", "đcm", "đcl",
    "địt", "đụ", "đĩ", "điếm", "lồn", "cặc",
    "chó", "khỉ", "lợn", "mọi",

    # ── Cụm ≥2 từ (giữ dấu cách) ──
    "địt mẹ", "đụ mẹ", "đụ má", "địt má", "đụ mẹ mày", "địt mẹ mày",
    "con chó", "lũ chó", "bọn chó", "chó chết", "chó má",
    "con khỉ", "lũ khỉ", "bọn khỉ",
    "con lợn", "lũ lợn", "bọn lợn",
    "con mọi", "thằng mọi", "bọn mọi", "lũ mọi", "mọi đen", "đám mọi", "đồ mọi", "mọi rợ",
    "con đĩ", "con điếm", "đồ đĩ", "đồ điếm", "đĩ rẻ", "đĩ chúa"
}

In [ ]:
data_list = []
with open('/data/npl/ICEK/VACNIC/data/backup/LLMs/official_test.jsonl', 'r', encoding='utf-8') as infile:
    i = 0
    for line in infile:
        item = json.loads(line)
        # Chỉ lấy translation và label
        data_list.append({
            'text': item['translation'],
            'label': item['label']
        })
    
df_data = pd.DataFrame(data_list)

In [4]:
df_data

,text,label
0,Bắt chước phong cách âm nhạc và cử chỉ của họ ...,Implicit HS
1,"David Allen Snyder, 26 tuổi, đã qua đời vào ng...",Non HS
2,Người Syria thì không biết quý trọng mạng sống...,Explicit HS
3,Dân tộc nào tồn tại hàng ngàn năm mà vẫn luôn ...,Non HS
4,"Nếu bạn có vấn đề với cô ấy, chắc là vì bạn gh...",Implicit HS
5,Có phải từng có vụ nổ lớn kiểu như burrito nổ ...,Implicit HS
6,Chắc như bụng sáu múi hả?,Non HS
7,Mày định làm gì mà để bọn Hồi giáo tràn vào đấ...,Explicit HS
8,Nếu tụi nó dám tấn công NATO thì NATO sẽ quét ...,Explicit HS
9,quan hệ với người da đỏ giống như lấy giấy nhá...,Implicit HS


In [ ]:
def is_explicit(text: str) -> bool:
    """
    Trả về True nếu text chứa bất kỳ slur nằm trong EXPLICIT_SLURS.
    Không phân biệt hoa thường; tách từ bằng regex \w+.
    """
    tokens = re.findall(r"\w+", text.lower())
    return any(tok in EXPLICIT_SLURS for tok in tokens)
 
def generate_output(input_text, prefix='hate-speech-detection'):
    prefixed_input_text = prefix + ': ' + input_text
    input_ids = tokenizer.encode(prefixed_input_text, return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=256)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text
 
def zero_shot_prediction_df(df, prefix='hate-speech-detection'):
    preds = []
    for text in tqdm(df['text'], desc="Đang sinh dự đoán"):
        output = generate_output(text, prefix)
        preds.append(output.lower().strip())

    df = df.copy()                           
    df['prediction'] = preds
    df['true_label'] = df['label'].map(
        lambda x: 'hate' if x in ('Implicit HS', 'Explicit HS') else 'clean'
    )

    def get_final_pred(row):
        label = row['label']        # 'Implicit HS' / 'Explicit HS' / 'non'
        pred  = row['prediction']   # 'hate' / 'clean'
        text  = row['text']

        if label == 'Implicit HS' and pred == 'hate':
            return 'Implicit HS'
        elif label == 'Explicit HS' and pred == 'hate':
            return 'Explicit HS'

        # hate thật nhưng mô hình bảo 'clean'
        elif label in ('Implicit HS', 'Explicit HS') and pred == 'clean':
            return 'Non HS'

        # non thật + mô hình clean
        elif label == 'Non HS' and pred == 'clean':
            return 'Non HS'

        # ---- TH còn lại: label = non & pred = hate ----
        else: return 'Explicit HS' if is_explicit(text) else 'Implicit HS'

    df['final_prediction'] = df.apply(get_final_pred, axis=1)
    return df

df_zero = zero_shot_prediction_df(df_data)

Đang sinh dự đoán:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
print(classification_report(df_zero['label'],df_zero['final_prediction']))

              precision    recall  f1-score   support

 Explicit HS       1.00      0.81      0.90        16
         IHS       0.00      0.00      0.00         0
 Implicit HS       1.00      0.07      0.13        14
      Non HS       0.52      0.85      0.64        20

    accuracy                           0.62        50
   macro avg       0.63      0.43      0.42        50
weighted avg       0.81      0.62      0.58        50



/home/npl/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/npl/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/npl/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
df

,text,label,flag,prediction
0,Bắt chước phong cách âm nhạc và cử chỉ của họ ...,Implicit HS,1,clean
1,"David Allen Snyder, 26 tuổi, đã qua đời vào ng...",Non HS,0,clean
2,Người Syria thì không biết quý trọng mạng sống...,Explicit HS,0,hate
3,Dân tộc nào tồn tại hàng ngàn năm mà vẫn luôn ...,Non HS,0,hate
4,"Nếu bạn có vấn đề với cô ấy, chắc là vì bạn gh...",Implicit HS,1,clean
5,Có phải từng có vụ nổ lớn kiểu như burrito nổ ...,Implicit HS,1,clean
6,Chắc như bụng sáu múi hả?,Non HS,0,clean
7,Mày định làm gì mà để bọn Hồi giáo tràn vào đấ...,Explicit HS,0,hate
8,Nếu tụi nó dám tấn công NATO thì NATO sẽ quét ...,Explicit HS,0,clean
9,quan hệ với người da đỏ giống như lấy giấy nhá...,Implicit HS,1,clean


In [20]:
mask_imp = df_zero['label'] == 'Implicit HS'
print("📊 Report riêng cho Implicit HS (dự đoán hate ↔ clean):")
print(classification_report(
        df_zero.loc[mask_imp, 'true_label'],   # true: hate / clean
        df_zero.loc[mask_imp, 'prediction']))

📊 Report riêng cho Implicit HS (dự đoán hate ↔ clean):
              precision    recall  f1-score   support

       clean       0.00      0.00      0.00       0.0
        hate       0.00      0.00      0.00       4.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0



/home/npl/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/npl/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/npl/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/npl/.local/lib/python3.9/site-packag